# OpenSearch 한글 형태소 분석기 통한 키워드 검색 
>이 노트북은,
> - SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.
> - SageMaker Notebook **`conda_python3`** 에서 테스트 되었습니다.


여기서는 OpenSearch 가 설치된 것을 가정하고, 한글 형태소 분석기의 사용하는 법을 알려 드립니다.

---

### [중요]
- 이 노트북은 Bedrock Titan Embedding Model 을 기본으로 사용합니다. KoSIMCSERoberta 를 세이지 메이커 엔드포인트로 사용하신다면 아래의 선수 조건을 확인하세요.

#### 선수조건 (KoSIMCSERoberta 사용시)
- 임베딩 모델의 세이지 메이커 엔드포인트가 액티브 된 상태를 가정 합니다.
    - 세이지 메이커 엔드포인트에 배포하기 위해서는 아래 노트북을 실행하시고, Endpoint Name 만을 복사 하시면 됩니다.
    - [KoSIMCSERoberta Embedding Model 배포](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/1-Lab01-Deploy-LLM/4.Kor-Embedding-Model.ipynb)
    - SageMaker Endpoint 에 대해서는 공식 개발자 문서를 참조하세요 --> [Create your endpoint and deploy your model](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html)
- 오픈 서치 서비스가 액티브 된 상태를 가정 합니다.


---
## Ref: 
- [Amazon OpenSearch Service로 검색 구현하기](https://catalog.us-east-1.prod.workshops.aws/workshops/de4e38cb-a0d9-4ffe-a777-bf00d498fa49/ko-KR/indexing/blog-reindex)
- [OpenSearch Python Client](https://opensearch.org/docs/1.3/clients/python-high-level/)
- [OpenSearch Match, Multi-Match, and Match Phrase Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-match-multi-match-and-match-phrase-queries/)
- OpenSearch Query 에서 Filter, Must, Should, Not Mush 에 대한 설명 입니다.
    - [OpenSearch Boolean Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-boolean-queries/#:~:text=Boolean%20queries%20are%20used%20to,as%20terms%2C%20match%20and%20query_string.)
- [OpenSearch Query Description (한글)](https://esbook.kimjmin.net/05-search)


## 1. 환경 세팅

In [1]:
import boto3
region = boto3.Session().region_name
opensearch = boto3.client('opensearch', region)

%store -r opensearch_user_id opensearch_user_password domain_name opensearch_domain_endpoint

try:
    opensearch_user_id
    opensearch_user_password
    domain_name
    opensearch_domain_endpoint
   
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Run 00_setup notebook first or Create Your Own OpenSearch Domain")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [2]:
%load_ext autoreload
%autoreload 2

### Bedrock Client 생성

In [3]:
import boto3
import os
import json
from botocore.config import Config
import botocore 
from pprint import pprint
from termcolor import colored

session = boto3.Session()

retry_config = Config(
    region_name=os.environ.get("AWS_DEFAULT_REGION", None),
    retries={
        "max_attempts": 10,
        "mode": "standard",
    },
)

# modelId = "anthropic.claude-instant-v1"  # (Change this to try different model versions)
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
accept = "application/json"
contentType = "application/json"

bedrock = boto3.client(service_name='bedrock')
boto3_bedrock = boto3.client(service_name='bedrock-runtime',config=retry_config)

model_list = bedrock.list_foundation_models()
result = [(fm_list["modelName"], fm_list["modelId"]) for fm_list in model_list["modelSummaries"] if fm_list['inferenceTypesSupported'] == ['ON_DEMAND']]
pprint(result)

[('Titan Text Large', 'amazon.titan-tg1-large'),
 ('Titan Text Embeddings v2', 'amazon.titan-embed-g1-text-02'),
 ('Titan Text G1 - Lite', 'amazon.titan-text-lite-v1'),
 ('Titan Text G1 - Express', 'amazon.titan-text-express-v1'),
 ('Titan Embeddings G1 - Text', 'amazon.titan-embed-text-v1'),
 ('Titan Multimodal Embeddings G1', 'amazon.titan-embed-image-v1'),
 ('Titan Image Generator G1', 'amazon.titan-image-generator-v1'),
 ('SDXL 0.8', 'stability.stable-diffusion-xl'),
 ('SDXL 0.8', 'stability.stable-diffusion-xl-v0'),
 ('SDXL 1.0', 'stability.stable-diffusion-xl-v1'),
 ('J2 Grande Instruct', 'ai21.j2-grande-instruct'),
 ('J2 Jumbo Instruct', 'ai21.j2-jumbo-instruct'),
 ('Jurassic-2 Mid', 'ai21.j2-mid'),
 ('Jurassic-2 Mid', 'ai21.j2-mid-v1'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra-v1'),
 ('Claude Instant', 'anthropic.claude-instant-v1'),
 ('Claude', 'anthropic.claude-v2:1'),
 ('Claude', 'anthropic.claude-v2'),
 ('Claude 3 Sonnet', 'anthropic.clau

## 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

### LLM 로딩 (Claude-v2.1)

In [4]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [5]:
llm_text = BedrockChat(
    model_id=modelId,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature" : 0,
        "top_k": 350,
        "top_p": 0.999
    }
)
llm_text

BedrockChat(client=<botocore.client.BedrockRuntime object at 0x7fd150914790>, region_name='us-west-2', model_id='anthropic.claude-3-sonnet-20240229-v1:0', model_kwargs={'anthropic_version': 'bedrock-2023-05-31', 'max_tokens': 4096, 'temperature': 0, 'top_k': 350, 'top_p': 0.999}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7fd18d0c6410>])

In [6]:
prompt1 = "나는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요."
messages = [
    HumanMessage(content=prompt1)
]

# messages = [
#     {"role": "user", "content": [{"type": "text", "text": prompt1}]},
# ]

response1 = llm_text.invoke(messages)

생명보험과 손해보험은 보험의 주요 유형으로 다음과 같은 차이점이 있습니다.

1. 보장 대상
- 생명보험은 사람의 생명과 관련된 위험을 보장합니다. 예를 들어 사망, 상해, 질병 등을 대상으로 합니다.
- 손해보험은 재산상의 손해나 배상책임을 보장합니다. 예를 들어 화재, 자동차사고, 배상책임 등을 대상으로 합니다.

2. 보험금 지급 사유
- 생명보험은 피보험자의 사망, 상해, 질병 등 인적 위험이 발생했을 때 보험금을 지급합니다.
- 손해보험은 재물의 손해나 법적 배상책임이 발생했을 때 실제 입은 손해를 보상합니다.

3. 보험기간
- 생명보험은 일반적으로 장기계약이며, 종신보험의 경우 피보험자 종신까지 보장됩니다.
- 손해보험은 단기계약이 일반적이며, 1년 만기로 갱신하는 경우가 많습니다.

4. 보험료 산정 기준
- 생명보험료는 피보험자의 나이, 건강상태, 가입금액 등에 따라 결정됩니다.
- 손해보험료는 보험대상의 가액, 위험률, 보상한도 등에 따라 결정됩니다.

요컨대 생명보험과 손해보험은 보장대상, 보험금 지급사유, 기간, 보험료 산정기준 등에서 차이가 있습니다.

In [7]:
prompt2 = "대상을 좀더 상세히 설명해 주세요"
messages = [
    HumanMessage(content=prompt1),
    response1,
    HumanMessage(content=prompt2),
]
response2 = llm_text.invoke(messages)

네, 생명보험과 손해보험의 보장 대상을 좀 더 자세히 설명드리겠습니다.

생명보험:
- 사람의 생명 자체를 보장합니다. 즉, 피보험자의 사망을 주요 보장사건으로 합니다.
- 상해, 질병, 장애 등 신체적 위험도 보장대상에 포함됩니다.
- 의료실비, 암, 치매 등 특정 질병도 보장합니다.
- 소득보상보험과 같이 피보험자의 노동능력 상실도 보장할 수 있습니다.

손해보험:
- 재물의 물리적 손해를 보장합니다. 예를 들어 화재, 풍수재 등으로 인한 주택/차량 등의 손해를 보상합니다. 
- 배상책임도 주요 보장대상입니다. 타인에게 손해를 입혀 법적 배상책임이 발생하면 보상해 줍니다.
- 기업을 대상으로 한 배상책임보험, 신용보험, 기계보험 등도 있습니다.
- 상해보험과 같이 사람의 신체적 상해도 보장대상이 될 수 있습니다.

요컨대 생명보험은 사람의 생명과 신체를, 손해보험은 재물의 손해와 배상책임을 주된 보장대상으로 합니다.

### Embedding 모델 선택

In [8]:
from langchain_community.embeddings import BedrockEmbeddings

llm_emb = BedrockEmbeddings(
    client=boto3_bedrock,
    # model_id="cohere.embed-multilingual-v3"
    model_id="amazon.titan-embed-g1-text-02"
)

-------------------

## 3. OpenSearch 벡터 Index 생성
### 선수 조건
- 랭체인 오프서처 참고 자료
    - [Langchain Opensearch](https://python.langchain.com/docs/integrations/vectorstores/opensearch)

### 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [9]:
from opensearchpy import OpenSearch, RequestsHttpConnection
http_auth = (opensearch_user_id, opensearch_user_password)
os_client = OpenSearch(
            hosts=[
                {'host': opensearch_domain_endpoint.replace("https://", ""),
                 'port': 443
                }
            ],
            http_auth=http_auth, # Master username, Master password,
            use_ssl=True,
            verify_certs=True,
            connection_class=RequestsHttpConnection
        )

### 형태소 분석기 사용하기
- 영어권의 문자들과 다르게 한글, 일본어, 중국어 등은 단순한 공백만으로는 좋은 검색 결과를 얻기 힘듭니다.
- 출시하고라는 단어를 출시, 출시하고 등 다양하게 검색하기 위해서는 형태소 분석기가 필요합니다.
- OpenSearch 에서는 2개의 한국어 analyer를 제공하고 있습니다.
    - 은전한잎 (seunjeon_tokenizer)
        - https://catalog.us-east-1.prod.workshops.aws/workshops/de4e38cb-a0d9-4ffe-a777-bf00d498fa49/ko-KR/indexing/stemming#
    - Nori (nori_tokenizer)
        - 설명: https://esbook.kimjmin.net/06-text-analysis/6.7-stemming/6.7.2-nori
    - Sample 코드에서는 "Nori"를 기반으로 진행합니다.

### [TIP]
- **token filter**
    - Amazon OpenSearch Service에서는 나만의 필터를 토크나이저와 함께 구성해서 사용할 수 있습니다. 
    - nori_tokenizer를 사용하되, 원하는 캐릭터 필터와 토큰 필터 조합을 구성할 수 있습니다.
    - 문장이 입력되면 캐릭터 필터(char_filter), 토크나이저(tokenizer), 토큰 필터(filter) 순으로 동작하게 됩니다.
    - 입력문장이 "<b>Start</b> 이천이십삼년 韓國" 이것일 경우, 
    - 아래 샘플 코드의 옵션으로 사용한다면 <b>Start</b>의 html 코드가 html_strip에 의해 처리되고, nori_tokenizer로 토큰화됩니다.
    - 그 후에 nori_number는 이천이십삼을 2023으로, nori_readingform은 韓國을 한국으로, lowercase는 Start를 start로 처리합니다.
    - **토큰필터 참고자료**
        - https://esbook.kimjmin.net/06-text-analysis/6.6-token-filter
        - https://opensearch.org/docs/latest/analyzers/token-filters/index/
    
- **discard_punctuation**
    - true(기본값), false가 있으며 문장부호 또는 구두점을 어떻게 다룰지에 대한 설정을 할 수 있습니다.
- **decompound_mode**
    - none, discard(기본값), mixed가 있으며 복합명사를 어떻게 다룰지에 대한 설정을 할 수 있습니다.
    - none: 복합명사를 분리하지 않고 하나의 토큰으로 저장합니다.
    - discard: 복합명사를 분리하여 토큰으로 저장합니다.
    - mixed: 복합명사를 분리하지 않은 토큰과 분리한 토큰을 모두 저장합니다.
- **신조어, 업무용어, 상표**
     - 노리 토크나이저는 [mecab-ko-dic](https://bitbucket.org/eunjeon/mecab-ko-dic/src/master/) 을 사용하고 있지만 때로는 신조어, 업무 용어, 상표 등을 위한 사용자 사전이 필요할 수 있습니다.
     - user_dictionary_rules를 이용해서 사용자 사전을 만들 수 있고, 아래에서 사용자 사전을 적용했을 때와 적용하지 않았을 때를 비교해 볼 수 있습니다.
- **동의어, 불용어**
    - Amazon OpenSearch Service에서는 Package를 이용하면 사용자사전, 동의어, 불용어를 관리할 수 있습니다. 자세한 사항은 [이곳](https://docs.aws.amazon.com/ko_kr/opensearch-service/latest/developerguide/custom-packages.html)을 참조해 주십시오. 
    - S3에 사용자 사전을 업로드 후, Package에 등록합니다. 그리고 등록된 텍스트 사전을 원하는 OpenSearch 도메인에 연결합니다.
    - 아래와 같이 user_dictionary에 analyzers/<Package ID> 를 이용해서 적용할 수 있습니다. 이 Package ID는 OpenSearch 콘솔 화면에서 업로드한 사전의 상세 페이지에서 확인할 수 있습니다.


### index 생성

In [10]:
index_name = "genai-demo-index-v1"
exists = os_client.indices.exists(index_name)

if exists:
    os_client.indices.delete(index=index_name)
    print("Index is deleted")
else:
    print("Index does not exist")

Index is deleted


In [11]:
## metadata, text, vector_field 의 네이밍은 langchain에서 지정된 이름
### model에 따라 dimension 사이즈 변경 필요 (Titan : 1536, Cohere : 1024)
import json

with open('index_body_simple.json', 'r') as f:
    index_body = json.load(f)

print(json.dumps(index_body, indent=2))


{
  "settings": {
    "index.knn": true,
    "index.knn.algo_param.ef_search": 512
  },
  "mappings": {
    "properties": {
      "metadata": {
        "properties": {
          "source": {
            "type": "keyword"
          },
          "type": {
            "type": "keyword"
          },
          "timestamp": {
            "type": "date"
          }
        }
      },
      "text": {
        "type": "text",
        "analyzer": "nori"
      },
      "vector_field": {
        "type": "knn_vector",
        "dimension": 1536,
        "method": {
          "engine": "faiss",
          "name": "hnsw",
          "parameters": {
            "ef_construction": 512,
            "m": 16
          },
          "space_type": "l2"
        }
      }
    }
  }
}


In [12]:
os_client.indices.create(index_name, body=index_body)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'genai-demo-index-v1'}

In [13]:
%%time
from langchain_community.vectorstores import OpenSearchVectorSearch

vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
)

CPU times: user 8.12 ms, sys: 0 ns, total: 8.12 ms
Wall time: 7.73 ms


## 4. 데이터 준비


### 한화생명 보험 약관 데이터 세트로 구현

In [14]:
import time
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PyPDFium2Loader


from langchain_core.documents import Document

# from llmsherpa.readers import LayoutPDFReader

In [16]:
import glob

data_path = './data/hanwhalife/*'
pdf_list = glob.glob(data_path)
pdf_list

['./data/hanwhalife/한화생명 간편가입 H튼튼 보장보험_20240101~.pdf',
 './data/hanwhalife/한화생명 간편가입 H튼튼 보장보험_20230601~.pdf',
 './data/hanwhalife/한화생명 간편가입 웰다잉 종신보험_20240101~.pdf',
 './data/hanwhalife/한화생명 The스마트한 일시납종신보험_20240101~.pdf',
 './data/hanwhalife/한화생명 The실속있는 간병치매보험_20231209~_1.pdf',
 './data/hanwhalife/한화생명 The스마트한 일시납종신보험_1912-061~072_20230916~.pdf',
 './data/hanwhalife/한화생명 간편가입 웰다잉 종신보험_20230901~.pdf',
 './data/hanwhalife/한화생명 간편가입 웰다잉 종신보험_20230916~.pdf',
 './data/hanwhalife/한화생명 The실속있는 간병치매보험_20240101~.pdf']

In [17]:
from multiprocessing.pool import ThreadPool
from multiprocessing import  Manager

import pdfplumber

In [18]:
import re

def prune_text(text, current_pdf_file):

    def replace_cid(match):
        print(f"Please check PDF file {current_pdf_file} : {match}")
        ascii_num = int(match.group(1))
        try:
            return chr(ascii_num)
        except:
            return ''  # In case of conversion failure, return empty string

    # Regular expression to find all (cid:x) patterns
    cid_pattern = re.compile(r'\(cid:(\d+)\)')
    pruned_text = re.sub(cid_pattern, replace_cid, text)
    return pruned_text

In [19]:
from datetime import datetime

def read_pdf(param):
    vector_db = param[0]
    current_pdf_file = param[1]
    print(f"current_pdf_file : {current_pdf_file}")
    docs = []
    source_name = current_pdf_file.split('/')[-1]
    type_name = source_name.split(' ')[-1].replace('.pdf', '')
    
    with pdfplumber.open(current_pdf_file) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:
                pruned_text = prune_text(page_text, current_pdf_file)
            else:
                pruned_text = ""
            if len(pruned_text) >= 20:  ## 임의로 20 이상인 sentence만 뽑도록 함
                chunk = Document(
                    page_content=pruned_text.replace('\n',' '),
                    metadata={
                        "source" : source_name,
                        "type": type_name,
                        "timestamp": datetime.now()
                    }
                )
                #print(f"chunk : {chunk}")
                docs.append(chunk)
    if len(docs) > 0 :
        vector_db.add_documents(docs)

In [20]:
manager = Manager()
result_dict = manager.dict()

# ml.m5.xlarge에서 multiprocessing으로 동작 확인
param = [(vector_db, current_pdf_file)for current_pdf_file in pdf_list]
# param = [(vector_db, current_pdf_file)for current_pdf_file in glob.glob("./data/hanwhalife/한화생명 The스마트한 일시납종신보험_20230916~.pdf")]
# param = [(vector_db, current_pdf_file)for current_pdf_file in glob.glob("./data/hanwhalife/한화생명 The실속있는 간병치매보험_20240101~.pdf")]

num_processes = len(pdf_list)%os.cpu_count()

if num_processes == 0 :
    num_processes = os.cpu_count() - 1

print(f"num of process : {num_processes}")

with ThreadPool(processes=num_processes) as pool:
    pool.map(read_pdf, param)
    pool.close()
    pool.join()

num of process : 9
current_pdf_file : ./data/hanwhalife/한화생명 간편가입 H튼튼 보장보험_20240101~.pdf
current_pdf_file : ./data/hanwhalife/한화생명 간편가입 H튼튼 보장보험_20230601~.pdf
current_pdf_file : ./data/hanwhalife/한화생명 간편가입 웰다잉 종신보험_20240101~.pdf
current_pdf_file : ./data/hanwhalife/한화생명 The스마트한 일시납종신보험_20240101~.pdf
current_pdf_file : ./data/hanwhalife/한화생명 The실속있는 간병치매보험_20231209~_1.pdf
current_pdf_file : ./data/hanwhalife/한화생명 The스마트한 일시납종신보험_1912-061~072_20230916~.pdf
current_pdf_file : ./data/hanwhalife/한화생명 간편가입 웰다잉 종신보험_20230901~.pdf
current_pdf_file : ./data/hanwhalife/한화생명 간편가입 웰다잉 종신보험_20230916~.pdf
current_pdf_file : ./data/hanwhalife/한화생명 The실속있는 간병치매보험_20240101~.pdf


### OpenSearch에 생성된 인덱스의 구성 확인

In [21]:
index_info = os_client.indices.get(index=index_name)
print(json.dumps(index_info, indent=2))

{
  "genai-demo-index-v1": {
    "aliases": {},
    "mappings": {
      "properties": {
        "metadata": {
          "properties": {
            "source": {
              "type": "keyword"
            },
            "timestamp": {
              "type": "date"
            },
            "type": {
              "type": "keyword"
            }
          }
        },
        "text": {
          "type": "text",
          "analyzer": "nori"
        },
        "vector_field": {
          "type": "knn_vector",
          "dimension": 1536,
          "method": {
            "engine": "faiss",
            "space_type": "l2",
            "name": "hnsw",
            "parameters": {
              "ef_construction": 512,
              "m": 16
            }
          }
        }
      }
    },
    "settings": {
      "index": {
        "replication": {
          "type": "DOCUMENT"
        },
        "number_of_shards": "5",
        "knn.algo_param": {
          "ef_search": "512"
        },
       

## 5. 어휘를 기반으로 한 전문(full-text) 검색 - Lexical Search

> query의 문장은 Nori 형태소 분석기에 의해 형태소로 분리됨
> OpenSearch에 입력된 (PDF에서 추출한) 문장(청크)들과 BM25 알고리즘을 기반으로 가장 유사한(score가 높은) 값부터 상위 (k=5)개의 결과가 나타남  

In [22]:
from opensearch_dsl import Search

In [23]:
def result_to_dataframe(response):
    import pandas as pd

    pd.set_option('display.max_columns', 150)
    pd.set_option('display.max_colwidth', None)

    result = []
    for res in response['hits']['hits']:
        # print(res.keys())
        result.append([res['_index'], round(res['_score'], 4), res['_source']['metadata']['type'], res['_source']['text']])
    df = pd.DataFrame(result, columns=['index_name', 'score', 'type', 'text'])
    return df.style.set_properties(**{'text-align': 'left'})

In [24]:
def query_lexical(query, filter=[], k=5):
    QUERY_TEMPLATE = {
        "size": k,
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": query
                        }
                    }
                ],
                "filter": filter
            }
        }
    }
    if len(filter) > 0:
        QUERY_TEMPLATE["query"]["bool"]["filter"].extend(filter)
    return QUERY_TEMPLATE

In [25]:
#query = "계약 전 의무 사항은 무엇인가요"
query = "금리연동형 상품은 무엇인가요?"

response_lexical_only = os_client.search(
    body=query_lexical(query),
    index=index_name
)

time_took_lexical_only = response_lexical_only['took']
print('검색에 걸린 시간: ', time_took_lexical_only, 'ms')

print("<<사용자 입력 쿼리 문장>>: ", query)

result_to_dataframe(response_lexical_only)

검색에 걸린 시간:  45 ms
<<사용자 입력 쿼리 문장>>:  금리연동형 상품은 무엇인가요?


,index_name,score,type,text
0,genai-demo-index-v1,18.470600,일시납종신보험_1912-061~072_20230916~,"Ⅱ. 약관 요약서 1. 보험계약의 개요 가입하신 보험상품의 보장내용과 특징을 확인해보세요. 한화생명 The스마트한 일시납종신보험 무배당 [보증비용부과형] 상품의 종목 : 보장성·종신보험 (1) 상품의 주요특징 예금자보호 금리연동형 예금자보호법에 따라 보험회사의 자산운용이익률, 1인당 최고 5천만원까지 시장금리 등에 따라 적립액 보장합니다. 다만, 보험계약자 적용이율이 변동되는 상품입 및 보험료 납부자가 법인인 니다. 경우에는 보호하지 않습니다. 보장성보험 한화생명 무배당 The스마트한 사망, 질병, 계약자에게 일시납종신보험 상해 등의 위험을 배당을 하지 않는 무배당 보장하는 보장성 상품입니다. 보험입니다. [보증비용부과형] 최저보증이율 해약환급금 보증 (해약환급금 보증형 가입시) 금리가 하락하더라도 적립시 적용되는 공시이율을 최저해약환급금은 회사가 정한 시산에 따라 향후 공시이율에 관계없이 일정 비율로 최저보증합니다. 계약 해지 시 지급하는 최저한도의 해약환급 금으로서, 예정적립액과 동일합니다. 15 / 235"
1,genai-demo-index-v1,18.470600,일시납종신보험_20240101~,"Ⅱ. 약관 요약서 1. 보험계약의 개요 가입하신 보험상품의 보장내용과 특징을 확인해보세요. 한화생명 The스마트한 일시납종신보험 무배당 [보증비용부과형] 상품의 종목 : 보장성·종신보험 (1) 상품의 주요특징 예금자보호 금리연동형 예금자보호법에 따라 보험회사의 자산운용이익률, 1인당 최고 5천만원까지 시장금리 등에 따라 적립액 보장합니다. 다만, 보험계약자 적용이율이 변동되는 상품입 및 보험료 납부자가 법인인 니다. 경우에는 보호하지 않습니다. 보장성보험 한화생명 무배당 The스마트한 사망, 질병, 계약자에게 일시납종신보험 상해 등의 위험을 배당을 하지 않는 무배당 보장하는 보장성 상품입니다. 보험입니다. [보증비용부과형] 최저보증이율 해약환급금 보증 (해약환급금 보증형 가입시) 금리가 하락하더라도 적립시 적용되는 공시이율을 최저해약환급금은 회사가 정한 시산에 따라 향후 공시이율에 관계없이 일정 비율로 최저보증합니다. 계약 해지 시 지급하는 최저한도의 해약환급 금으로서, 예정적립액과 동일합니다. 15 / 249"
2,genai-demo-index-v1,15.899300,일시납종신보험_1912-061~072_20230916~,"(2) 소비자가 반드시 알아두어야 할 상품의 주요 특성 보장성보험 ① 이 보험은 사망 보장을 주목적으로 하는 보장성보험이며, 저축이나 연금수령을 목적으로 가입하시기에 적합하지 않습니다. ② 만기 또는 중도해지 시 납입한 보험료보다 환급금이 적을 수 있습니다. ③ 사망보험금의 일부 또는 전부를 해지하여 연금으로 전환할 수 있으나, 처음부터 연금보험에 가입한 경우보다 연금액이 적습니다. [민원 사례] A씨는 저축을 주된 목적으로 종신보험을 가입하였으나 알고보니 사망 및 질병 보장을 주목적으로 하는 보장성보험임에 불만 ▶ 종신보험 등 보장성보험은 저축이나 노후에 연금 수령을 목적으로 가입하시기에는 적합하지 않으며 또한 중도에 해지하면 손해를 볼 수 있음 금리연동형보험 ① 이 보험의 적립액 산출에 적용되는 이율은 매월 변동됩니다. ② 동 이율은 납입한 주계약(또는 적립) 보험료에서 계약체결·유지관리에 필요한 경비 및 위험보장을 위한 보험료를 차감한 금액에 대해서만 적용됩니다. ③ 이 보험의 최저보증이율*은 3년이내에는 연복리 1.0%, 3년초과 5년이내에는 연복리 0.75%, 5년을 초과하는 경우에는 연복리 0.5%입니다. *자산운용이익률, 시장금리가 하락하는 경우 회사가 보증하는 적용이율의 최저한도 유니버셜보험 이 보험은 적립액 인출 등을 하는 경우 계약이 조기에 실효될 수 있습니다. * 해약환급금에서 월대체보험료(계약 유지관리 등에 소요되는 경비, 위험 보장을 위한 보험료)를 매 월 공제 [민원 사례] A씨는 일시납보험료를 하면 보장을 계속 받을 수 있다고 생각하고 ○○○○ 유니버설종신보험에 가입하였으나, 4년이 지난 후 보험회사에서 보험료 납입을 독촉하는 안내를 받음 ▶ 보험회사는 유니버설보험의 경우 기본보험료 납입 이후 중도인출 등으로 인해, 주계약 해약환급금에서 월대체보험료를 충당할 수 없게 된 경우 계약이 해지될 수 있고, 계약을 유지하기 위해서는 보험료 추가 납입이 필요함을 안내 예금자보호제도에 ① 이 보험은 예금자보호법에 따라 예금보험공사가 보호합니다. 관한 사항 ② 예금자보호 한도는 본 보험회사에 있는 귀하의 모든 예금보호 대상 금융상품의 해약환급금(또는 만기시 보험금이나 사고보험금)과 기타 지급금을 합하여 1인당 “최고 5천만원”이며, 5천만원을 초과하는 나머지 금액은 보호하지 않습니다. ③ 또한, 보험계약자 및 보험료 납부자가 법인이면 보호되지 않습니다. 19 / 235"
3,genai-demo-index-v1,14.382200,일시납종신보험_20240101~,"(2) 소비자가 반드시 알아두어야 할 상품의 주요 특성 보장성보험 ① 이 보험은 사망 보장을 주목적으로 하는 보장성보험이며, 저축이나 연금수령을 목적으로 가입하시기에 적합하지 않습니다. ② 만기 또는 중도해지 시 납입한 보험료보다 환급금이 적을 수 있습니다. ③ 사망보험금의 일부 또는 전부를 해지하여 연금으로 전환할 수 있으나, 처음부터 연금보험에 가입한 경우보다 연금액이 적습니다. [민원 사례] A씨는 저축을 주된 목적으로 종신보험을 가입하였으나 알고보니 사망 및 질병 보장을 주목적으로 하는 보장성보험임에 불만 ▶ 종신보험 등 보장성보험은 저축이나 노후에 연금 수령을 목적으로 가입하시기에는 적합하지 않으며 또한 중도에 해지하면 손해를 볼 수 있음 금리연동형보험 ① 이 보험의 적립액 산출에 적용되는 이율은 매월 변동됩니다. ② 동 이율은 납입한 주계약(또는 적립) 보험료에서 계약체결·유지관리에 필요한 경비 및 위험보장을 위한 보험료를 차감한 금액에 대해서만 적용됩니다. ③ 이 보험의 최저보증이율*은 3년이내에는 연복리 1.0%, 3년초과 5년이내에는 연복리 0.75%, 5년을 초과하는 경우에는 연복리 0.5%입니다. *자산운용이익률, 시장금리가 하락하는 경우 회사가 보증하는 적용이율의 최저한도 유니버셜보험 이 보험은 적립액 인출 등을 하는 경우 계약이 조기에 실효될 수 있습니다. * 해약환급금에서 월대체보험료(계약 유지관리 등에 소요되는 경비, 위험 보장을 위한 보험료)를 매 월 공제 [민원 사례] A씨는 일시납보험료를 하면 보장을 계속 받을 수 있다고 생각하고 ○○○○ 유니버설종신보험에 가입하였으나, 4년이 지난 후 보험회사에서 보험료 납입을 독촉하는 안내를 받음 ▶ 보험회사는 유니버설보험의 경우 기본보험료 납입 이후 중도인출 등으로 인해, 주계약 해약환급금에서 월대체보험료를 충당할 수 없게 된 경우 계약이 해지될 수 있고, 계약을 유지하기 위해서는 보험료 추가 납입이 필요함을 안내 예금자보호제도에 ① 이 보험은 예금자보호법에 따라 예금보험공사가 보호합니다. 관한 사항 ② 예금자보호 한도는 본 보험회사에 있는 귀하의 모든 예금보호 대상 금융상품의 해약환급금(또는 만기시 보험금이나 사고보험금)과 기타 지급금을 합하여 1인당 “최고 5천만원”이며, 5천만원을 초과하는 나머지 금액은 보호하지 않습니다. ③ 또한, 보험계약자 및 보험료 납부자가 법인이면 보호되지 않습니다. 19 / 249"
4,genai-demo-index-v1,11.434800,간병치매보험_20231209~_1,"Ⅱ. 약관 요약서 1. 보험계약의 개요 가입하신 보험상품의 보장내용과 특징을 확인해보세요. 한화생명 The실속있는 간병치매보험 무배당 상품의 종목: 보장성보험, 치매보험 (1) 상품의 주요특징 (1) 상품의 주요특징 금리확정형 예금자보호 보험료 및 적립금 산출에 예금자보호법에 따라 적용되는 이율이 1인당 최고 5천만원까지 보험기간 동안 고정되어 보장합니다. 변동이 없습니다. 보장성보험 무배당 사망, 

### 자연어 쿼리 분석 

In [26]:
response = os_client.indices.analyze(
    index=index_name,
    body={
        "text": query,
        "analyzer": "nori" 
    }
)

print(json.dumps(response, indent=2,  ensure_ascii=False))

{
  "tokens": [
    {
      "token": "금리",
      "start_offset": 0,
      "end_offset": 2,
      "type": "word",
      "position": 0
    },
    {
      "token": "연동",
      "start_offset": 2,
      "end_offset": 4,
      "type": "word",
      "position": 1
    },
    {
      "token": "상품",
      "start_offset": 6,
      "end_offset": 8,
      "type": "word",
      "position": 3
    },
    {
      "token": "무엇",
      "start_offset": 10,
      "end_offset": 12,
      "type": "word",
      "position": 5
    },
    {
      "token": "이",
      "start_offset": 12,
      "end_offset": 15,
      "type": "word",
      "position": 6
    }
  ]
}


위와 같이 쿼리로 작성된 자연어 문장이 token화 되어 검색됩니다.

### 결과값에 매칭되는 Term 확인

BM25 알고리즘에 의해 스코어링이 되고, PDF에서 추출된 문장(chunk)에서 어떤 term들과 매칭되었는지를 확인하려면 아래의 코드를 실행합니다. 아래 쿼리는 결과값에 매칭되는 term을 강조할 수 있도록 html 태그를 추가합니다.

In [27]:
def query_lexical_with_highlight(query, filter=[], k=5):
    QUERY_TEMPLATE = {
        "size": k,
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": query
                        }
                    }
                ],
                "filter": filter
            }
        },
        "highlight": {
            "pre_tags": [
                "<span style='color:red'>"
            ],
            "post_tags": [
                "</span>"
            ],
            "fields": {
                "text": {}
            }
        }
    }
    if len(filter) > 0:
        QUERY_TEMPLATE["query"]["bool"]["filter"].extend(filter)
    return QUERY_TEMPLATE

response_lexical_with_highlight = os_client.search(
    body=query_lexical_with_highlight(query),
    index=index_name
)


In [28]:
from IPython.display import HTML

import pandas as pd
temp_arr = []

for res in response_lexical_with_highlight['hits']['hits']:
            # result.append([res['_index'], round(res['_score'], 4), res['_source']['metadata']['type'], res['_source']['text']])

    temp_arr.append([res['_score'], res['highlight']['text']])

# print("---------- html 태그가 포함된 결과 ------------")
# print(temp_arr)
# print("-------------------------------------------")
    
#df = pd.DataFrame(temp_arr)
df = pd.DataFrame(temp_arr, columns=['score', '각 문서(chuck) 내에서 매칭된 부분'])

print("<<사용자 입력 쿼리 문장>>: ", query)

HTML(df.to_html(escape=False))


<<사용자 입력 쿼리 문장>>:  금리연동형 상품은 무엇인가요?


,score,각 문서(chuck) 내에서 매칭된 부분
0,18.470612,"[보험계약의 개요 가입하신 보험상품의 보장내용과 특징을 확인해보세요., 한화생명 The스마트한 일시납종신보험 무배당 [보증비용부과형] 상품의 종목 : 보장성·종신보험 (1) 상품의 주요특징 예금자보호 금리연동형 예금자보호법에 따라 보험회사의 자산운용이익률, , 1인당 최고 5천만원까지 시장금리 등에 따라 적립액 보장합니다., 다만, 보험계약자 적용이율이 변동되는 상품입 및 보험료 납부자가 법인인 니다. 경우에는 보호하지 않습니다., 보장성보험 한화생명 무배당 The스마트한 사망, 질병, 계약자에게 일시납종신보험 상해 등의 위험을 배당을 하지 않는 무배당 보장하는 보장성 상품입니다. 보험입니다.]"
1,18.470612,"[보험계약의 개요 가입하신 보험상품의 보장내용과 특징을 확인해보세요., 한화생명 The스마트한 일시납종신보험 무배당 [보증비용부과형] 상품의 종목 : 보장성·종신보험 (1) 상품의 주요특징 예금자보호 금리연동형 예금자보호법에 따라 보험회사의 자산운용이익률, , 1인당 최고 5천만원까지 시장금리 등에 따라 적립액 보장합니다., 다만, 보험계약자 적용이율이 변동되는 상품입 및 보험료 납부자가 법인인 니다. 경우에는 보호하지 않습니다., 보장성보험 한화생명 무배당 The스마트한 사망, 질병, 계약자에게 일시납종신보험 상해 등의 위험을 배당을 하지 않는 무배당 보장하는 보장성 상품입니다. 보험입니다.]"
2,15.899325,"[(2) 소비자가 반드시 알아두어야 할 상품의 주요 특성 보장성보험 ① 이 보험은 사망 보장을 주목적으로 하는 보장성보험이며, 저축이나 연금수령을 목적으로 가입하시기에 적합하지 않습니다, 수령을 목적으로 가입하시기에는 적합하지 않으며 또한 중도에 해지하면 손해를 볼 수 있음 금리연동형보험 ① 이 보험의 적립액 산출에 적용되는 이율은 매월 변동됩니다. ② 동 이율은, *자산운용이익률, 시장금리가 하락하는 경우 회사가 보증하는 적용이율의 최저한도 유니버셜보험 이 보험은 적립액 인출 등을 하는 경우 계약이 조기에 실효될 수 있습니다. * 해약환급금에서, 관한 사항 ② 예금자보호 한도는 본 보험회사에 있는 귀하의 모든 예금보호 대상 금융상품의 해약환급금(또는 만기시 보험금이나 사고보험금)과 기타 지급금을 합하여 1인당 “최고 5천만원, ”이며, 5천만원을 초과하는 나머지 금액은 보호하지 않습니다. ③ 또한, 보험계약자 및 보험료 납부자가 법인이면 보호되지 않습니다. 19]"
3,14.382164,"[(2) 소비자가 반드시 알아두어야 할 상품의 주요 특성 보장성보험 ① 이 보험은 사망 보장을 주목적으로 하는 보장성보험이며, 저축이나 연금수령을 목적으로 가입하시기에 적합하지 않습니다, 수령을 목적으로 가입하시기에는 적합하지 않으며 또한 중도에 해지하면 손해를 볼 수 있음 금리연동형보험 ① 이 보험의 적립액 산출에 적용되는 이율은 매월 변동됩니다. ② 동 이율은, *자산운용이익률, 시장금리가 하락하는 경우 회사가 보증하는 적용이율의 최저한도 유니버셜보험 이 보험은 적립액 인출 등을 하는 경우 계약이 조기에 실효될 수 있습니다. * 해약환급금에서, 관한 사항 ② 예금자보호 한도는 본 보험회사에 있는 귀하의 모든 예금보호 대상 금융상품의 해약환급금(또는 만기시 보험금이나 사고보험금)과 기타 지급금을 합하여 1인당 “최고 5천만원, ”이며, 5천만원을 초과하는 나머지 금액은 보호하지 않습니다. ③ 또한, 보험계약자 및 보험료 납부자가 법인이면 보호되지 않습니다. 19]"
4,11.434752,"[보험계약의 개요 가입하신 보험상품의 보장내용과 특징을 확인해보세요., 한화생명 The실속있는 간병치매보험 무배당 상품의 종목: 보장성보험, 치매보험 (1) 상품의 주요특징 (1) 상품의 주요특징 금리확정형 예금자보호 보험료 및 적립금 산출에 예금자보호법에, 보장성보험 무배당 사망, 질병, 계약자에게 한화생명 상해 등의 위험을 배당을 하지 않는 The실속있는 보장하는 보장성 상품입니다. 보험입니다., 미지급형 보험료 납입기간 중 경도, 중등도, 중증치매, 계약이 해지될 경우 말기치매 진단자금 및 간병자금을 해약환급금을 지급하지 않는 대신 보장하는 ‘표준형’보다 낮은 보험료로 보험상품입니다, 가입할 수 있는 상품입니다. 12]"


참고> hightlight 요청은 모든 결과를 return하지 않고, 매칭되는 부분만 중점적으로 보여줍니다. 위의 코드에서는 검색에 매칭되는 term의 앞 뒤로 \<span style='color:red'>과 \</span>태그로 눈에 들어오도록 표기했습니다.

## 6. Filter 활용
- document내 metadata를 활용하여 search space를 줄일 수 있다.
- 특히 filter의 경우 search 전에 수행되기 때문에, 검색 속도 향상을 기대할 수 있다
- syntax
    - filter=[{"term"**[고정]**: {"metadata.source"**[메타데이터 이름, 혹은 메타데이터 아니여도 상관없음]**: "신한은행"**[조건명]**}},]
    - list 형식으로 복수개 filter 설정 가능

In [29]:
filter = [
    {"term": {"metadata.source": "한화생명 The실속있는 간병치매보험_20240101~.pdf"}}
]

response = os_client.search(
    body=query_lexical(query, filter),
    index=index_name
)
result_to_dataframe(response)

,index_name,score,type,text
0,genai-demo-index-v1,10.301800,간병치매보험_20240101~,"Ⅱ. 약관 요약서 1. 보험계약의 개요 가입하신 보험상품의 보장내용과 특징을 확인해보세요. 한화생명 The실속있는 간병치매보험 무배당 상품의 종목: 보장성보험, 치매보험 (1) 상품의 주요특징 (1) 상품의 주요특징 금리확정형 예금자보호 보험료 및 적립금 산출에 예금자보호법에 따라 적용되는 이율이 1인당 최고 5천만원까지 보험기간 동안 고정되어 보장합니다. 변동이 없습니다. 보장성보험 무배당 사망, 질병, 계약자에게 한화생명 상해 등의 위험을 배당을 하지 않는 The실속있는 보장하는 보장성 상품입니다. 보험입니다. 간병치매보험 무배당 치매보험 해약환급금 미지급형 보험료 납입기간 중 경도, 중등도, 중증치매, 계약이 해지될 경우 말기치매 진단자금 및 간병자금을 해약환급금을 지급하지 않는 대신 보장하는 ‘표준형’보다 낮은 보험료로 보험상품입니다. 가입할 수 있는 상품입니다. 12 / 462"
1,genai-demo-index-v1,8.081300,간병치매보험_20240101~,"4. 상황별 목차 안내 계약의 계약에서 쓰이는 제2조(용어의 정의) 체결 용어를 알고 싶어요 제14조(계약 전 알릴 의무) 계약 전 알릴 의무는 무엇인가요 제15조(계약 전 알릴 의무 위반의 효과) 보장이 언제 시작되나요 제24조(제1회 보험료 및 회사의 보장개시) 청약을 철회하고 싶어요 제18조(청약의 철회) 계약을 취소 할 수 있나요 제19조(약관교부 및 설명의무 등) 계약이 무효가 될 수 있나요 제20조(계약의 무효) 보험료의 보험료 납입면제 제6조(보험금 지급에 관한 세부규정) 납입 사유를 알고 싶어요 보험료 납입최고(독촉)이 제27조(보험료의 납입이 연체되는 경우 무엇인가요 납입최고(독촉)와 계약의 해지) 제28조(보험료의 납입연체로 인하여 해지된 계약을 살리고 싶어요 해지된 계약의 부활(효력회복)) 보험금의 제5조(보험금의 지급사유), 보험금을 받을 수 있는지 궁금해요 지급 제7조(보험금을 지급하지 않는 사유) 제9조(보험금 등의 청구), 보험금은 언제 지급되나요 제10조(보험금 등의 지급절차) 8 / 462"
2,genai-demo-index-v1,6.940900,간병치매보험_20240101~,"하 ‘공시이율’이라 합니다)로 합니다. 공시이율의 최저보증이율은 이 특약의 계약일로부터 3년 이내에는 연복리 1.0%, 3년초과 5년 이내에는 연복리 0.75%, 5년을 초과하는 경우에는 연복리 0.5%를 적용합니다. ② 제1항의 공시이율은 이 보험의 사업방법서에서 정하는 바에 따라 운용자산이익률과 객관적인 외부지 표금리를 가중평균하여 산출된 공시기준이율에서 향후 예상수익 등을 고려한 조정률을 가감하여 결정합 니다. ③ 회사는 제1항 내지 제2항에서 정한 공시이율 및 산출방법 등을 매월 회사의 인터넷 홈페이지 등을 통해 공시합니다. 【 공시이율 】 보험회사가 장래 보험금 지급을 위하여 납입보험료의 일정 부분을 적립해 가는데, 이때 적용하는 이 율을 의미합니다. 이 보험의 공시이율은 회사의 운용자산이익률, 객관적인 외부지표금리(국고채 수익 률 등 시중금리) 및 향후 운용수익률 예측분을 혼합하여 일정기간마다 산출하여 공시하고 있습니다. 제 5 조 보험금 등의 청구 ① 보험수익자 또는 계약자는 다음의 서류를 제출하고 보험금 또는 계약자적립액을 청구하여야 합니다. 1. 청구서(회사양식) 2. 사고증명서(사망진단서 등) 3. 신분증(주민등록증이나 운전면허증 등 사진이 붙은 정부기관 발행 신분증, 본인이 아닌 경우에는 본인의 인감증명서 또는 본인서명사실확인서 포함) 4. 기타 보험수익자 또는 계약자가 보험금 수령 또는 계약자적립액 청구에 필요하여 제출하는 서류 【계약자적립액】 장래의 보험금, 해약환급금 등을 지급하기 위하여 계약자가 납입한 보험료 중 일정액을 기준으로 산 출방법서에서 정한 방법에 따라 계산한 금액 ② 제1항에도 불구하고 회사는 보험수익자가 의식불명 등 의사무능력상태인 경우 성년후견인이 보험수 익자를 대리하여 제1항 각호의 서류와 다음의 서류를 제출하고 보험금을 청구하도록 요구할 수 있습니 다. 1. 보험수익자가 예금주인 예금 통장 사본 2. 성년후견인의 보험금지급 동의서, 인감증명서, 인감도장 3. 보험수익자 기준 성년 후견 등기사항 증명서 ③ 제1항 제2호의 사고증명서는 의료법 제3조(의료기관)에서 규정한 국내의 병원이나 의원 또는 국외의 의료관련법에서 정한 의료기관에서 발급한 것이어야 합니다. 제 6 조 보험금 등의 지급절차 ① 회사는 제5조(보험금 등의 청구)에 정한 서류를 접수한 때에는 접수증을 드리고 휴대전화 문자메시지 또는 전자우편 등으로도 송부하며, 그 서류를 접수한 날부터 3영업일 이내에 보험금 또는 계약자적립액 을 지급합니다. 다만, 보험금 및 계약자적립액 지급사유의 조사나 확인이 필요한 때에는 접수 후 10영업 일 이내에 보험금 및 계약자적립액을 지급합니다. ② 회사는 제3조(보험금의 지급사유)에 해당하는 연금의 지급시기가 되면 지급시기 7일 이전에 그 사유 와 회사가 지급하여야 할 금액을 계약자 또는 보험수익자에게 알려드리며, 제1항에 따라 보험금 또는 계약자적립액을 지급할 때 지급일까지의 기간에 대한 이자는 ‘보험금을 지급할 때의 적립이율 계산’(별 표 2 참조)과 같이 계산합니다. 제 3 관 특약의 성립과 유지 제 7 조 특약의 성립 ① 이 특약은 계약자의 청약(請約)과 회사의 승낙(承諾)으로 전환전계약에 부가하여 이루어 집니다. ② 계약자는 이 특약을 청약할 때 종신연금형, 확정기간연금형 또는 상속연금형 중 한 가지를 선택하여 370 / 462"
3,genai-demo-index-v1,5.091200,간병치매보험_20240101~,"금융소비자 중 대통령령으로 정하는 자가 일반금융소비자와 같은 대우를 받겠다는 의사를 금융상 품판매업자 또는 금융상품자문업자(이하 “금융상품판매업자등”이라 한다)에게 서면으로 통지하는 경우 금융상품판매업자등은 정당한 사유가 있는 경우를 제외하고는 이에 동의하여야 하며, 금융상 품판매업자등이 동의한 경우에는 해당 금융소비자는 일반금융소비자로 본다. 가. 국가 나. 「한국은행법」에 따른 한국은행 다. 대통령령으로 정하는 금융회사 라. 「자본시장과 금융투자업에 관한 법률」 제9조제15항제3호에 따른 주권상장법인(투자성 상품 중 대통령령으로 정하는 금융상품계약체결등을 할 때에는 전문금융소비자와 같은 대우를 받겠다는 의사를 금융상품판매업자등에게 서면으로 통지하는 경우만 해당한다) 마. 그 밖에 금융상품의 유형별로 대통령령으로 정하는 자 10. “일반금융소비자”란 전문금융소비자가 아닌 금융소비자를 말한다. 제42조(소액분쟁사건에 관한 특례) 조정대상기관은 다음 각 호의 요건 모두를 총족하는 분쟁사건(이하 “소액분쟁사건”이라 한다)에 대하 여 조정절차가 개시된 경우에는 제36조제6항에 따라 조정안을 제시받기 전에는 소를 제기할 수 없다. 다만, 제36조제3항에 따라 서면통지를 받거나 제36조제5항에서 정한 기간 내에 조정안을 제시받지 못 한 경우에는 그러하지 아니하다. 1. 일반금융소비자가 신청한 사건일 것 2. 조정을 통하여 주장하는 권리나 이익의 가액이 2천만원 이내에서 대통령령으로 정하는 금액 이하 일 것 제47조(위법계약의 해지) ① 금융소비자는 금융상품판매업자등이 제17조제3항, 제18조제2항, 제19조제1항ㆍ제3항, 제20조제1항 또는 제21조를 위반하여 대통령령으로 정하는 금융상품에 관한 계약을 체결한 경우 5년 이내의 대 통령령으로 정하는 기간 내에 서면등으로 해당 계약의 해지를 요구할 수 있다. 이 경우 금융상품 판매업자등은 해지를 요구받은 날부터 10일 이내에 금융소비자에게 수락여부를 통지하여야 하며, 거절할 때에는 거절사유를 함께 통지하여야 한다. ② 금융소비자는 금융상품판매업자등이 정당한 사유 없이 제1항의 요구를 따르지 않는 경우 해당 계 약을 해지할 수 있다. ③ 제1항 및 제2항에 따라 계약이 해지된 경우 금융상품판매업자등은 수수료, 위약금 등 계약의 해지 와 관련된 비용을 요구

In [30]:
filter = [
    {"term": {"metadata.source": "한화생명 The실속있는 간병치매보험_20240101~.pdf"}},
    {"term": {"metadata.type": "간병치매보험_20240101~"}},
]

response = os_client.search(
    body=query_lexical(query, filter),
    index=index_name
)
print('이전에 필터를 넣치 않고 검색에 걸린 시간: ', time_took_lexical_only, 'ms')
print('지금 필터를 넣고 검색에 걸린 시간: ', response['took'], 'ms')

print("<<사용자 입력 쿼리 문장>>: ", query)

result_to_dataframe(response)

이전에 필터를 넣치 않고 검색에 걸린 시간:  45 ms
지금 필터를 넣고 검색에 걸린 시간:  9 ms
<<사용자 입력 쿼리 문장>>:  금리연동형 상품은 무엇인가요?


,index_name,score,type,text
0,genai-demo-index-v1,10.301800,간병치매보험_20240101~,"Ⅱ. 약관 요약서 1. 보험계약의 개요 가입하신 보험상품의 보장내용과 특징을 확인해보세요. 한화생명 The실속있는 간병치매보험 무배당 상품의 종목: 보장성보험, 치매보험 (1) 상품의 주요특징 (1) 상품의 주요특징 금리확정형 예금자보호 보험료 및 적립금 산출에 예금자보호법에 따라 적용되는 이율이 1인당 최고 5천만원까지 보험기간 동안 고정되어 보장합니다. 변동이 없습니다. 보장성보험 무배당 사망, 질병, 계약자에게 한화생명 상해 등의 위험을 배당을 하지 않는 The실속있는 보장하는 보장성 상품입니다. 보험입니다. 간병치매보험 무배당 치매보험 해약환급금 미지급형 보험료 납입기간 중 경도, 중등도, 중증치매, 계약이 해지될 경우 말기치매 진단자금 및 간병자금을 해약환급금을 지급하지 않는 대신 보장하는 ‘표준형’보다 낮은 보험료로 보험상품입니다. 가입할 수 있는 상품입니다. 12 / 462"
1,genai-demo-index-v1,8.081300,간병치매보험_20240101~,"4. 상황별 목차 안내 계약의 계약에서 쓰이는 제2조(용어의 정의) 체결 용어를 알고 싶어요 제14조(계약 전 알릴 의무) 계약 전 알릴 의무는 무엇인가요 제15조(계약 전 알릴 의무 위반의 효과) 보장이 언제 시작되나요 제24조(제1회 보험료 및 회사의 보장개시) 청약을 철회하고 싶어요 제18조(청약의 철회) 계약을 취소 할 수 있나요 제19조(약관교부 및 설명의무 등) 계약이 무효가 될 수 있나요 제20조(계약의 무효) 보험료의 보험료 납입면제 제6조(보험금 지급에 관한 세부규정) 납입 사유를 알고 싶어요 보험료 납입최고(독촉)이 제27조(보험료의 납입이 연체되는 경우 무엇인가요 납입최고(독촉)와 계약의 해지) 제28조(보험료의 납입연체로 인하여 해지된 계약을 살리고 싶어요 해지된 계약의 부활(효력회복)) 보험금의 제5조(보험금의 지급사유), 보험금을 받을 수 있는지 궁금해요 지급 제7조(보험금을 지급하지 않는 사유) 제9조(보험금 등의 청구), 보험금은 언제 지급되나요 제10조(보험금 등의 지급절차) 8 / 462"
2,genai-demo-index-v1,6.940900,간병치매보험_20240101~,"하 ‘공시이율’이라 합니다)로 합니다. 공시이율의 최저보증이율은 이 특약의 계약일로부터 3년 이내에는 연복리 1.0%, 3년초과 5년 이내에는 연복리 0.75%, 5년을 초과하는 경우에는 연복리 0.5%를 적용합니다. ② 제1항의 공시이율은 이 보험의 사업방법서에서 정하는 바에 따라 운용자산이익률과 객관적인 외부지 표금리를 가중평균하여 산출된 공시기준이율에서 향후 예상수익 등을 고려한 조정률을 가감하여 결정합 니다. ③ 회사는 제1항 내지 제2항에서 정한 공시이율 및 산출방법 등을 매월 회사의 인터넷 홈페이지 등을 통해 공시합니다. 【 공시이율 】 보험회사가 장래 보험금 지급을 위하여 납입보험료의 일정 부분을 적립해 가는데, 이때 적용하는 이 율을 의미합니다. 이 보험의 공시이율은 회사의 운용자산이익률, 객관적인 외부지표금리(국고채 수익 률 등 시중금리) 및 향후 운용수익률 예측분을 혼합하여 일정기간마다 산출하여 공시하고 있습니다. 제 5 조 보험금 등의 청구 ① 보험수익자 또는 계약자는 다음의 서류를 제출하고 보험금 또는 계약자적립액을 청구하여야 합니다. 1. 청구서(회사양식) 2. 사고증명서(사망진단서 등) 3. 신분증(주민등록증이나 운전면허증 등 사진이 붙은 정부기관 발행 신분증, 본인이 아닌 경우에는 본인의 인감증명서 또는 본인서명사실확인서 포함) 4. 기타 보험수익자 또는 계약자가 보험금 수령 또는 계약자적립액 청구에 필요하여 제출하는 서류 【계약자적립액】 장래의 보험금, 해약환급금 등을 지급하기 위하여 계약자가 납입한 보험료 중 일정액을 기준으로 산 출방법서에서 정한 방법에 따라 계산한 금액 ② 제1항에도 불구하고 회사는 보험수익자가 의식불명 등 의사무능력상태인 경우 성년후견인이 보험수 익자를 대리하여 제1항 각호의 서류와 다음의 서류를 제출하고 보험금을 청구하도록 요구할 수 있습니 다. 1. 보험수익자가 예금주인 예금 통장 사본 2. 성년후견인의 보험금지급 동의서, 인감증명서, 인감도장 3. 보험수익자 기준 성년 후견 등기사항 증명서 ③ 제1항 제2호의 사고증명서는 의료법 제3조(의료기관)에서 규정한 국내의 병원이나 의원 또는 국외의 의료관련법에서 정한 의료기관에서 발급한 것이어야 합니다. 제 6 조 보험금 등의 지급절차 ① 회사는 제5조(보험금 등의 청구)에 정한 서류를 접수한 때에는 접수증을 드리고 휴대전화 문자메시지 또는 전자우편 등으로도 송부하며, 그 서류를 접수한 날부터 3영업일 이내에 보험금 또는 계약자적립액 을 지급합니다. 다만, 보험금 및 계약자적립액 지급사유의 조사나 확인이 필요한 때에는 접수 후 10영업 일 이내에 보험금 및 계약자적립액을 지급합니다. ② 회사는 제3조(보험금의 지급사유)에 해당하는 연금의 지급시기가 되면 지급시기 7일 이전에 그 사유 와 회사가 지급하여야 할 금액을 계약자 또는 보험수익자에게 알려드리며, 제1항에 따라 보험금 또는 계약자적립액을 지급할 때 지급일까지의 기간에 대한 이자는 ‘보험금을 지급할 때의 적립이율 계산’(별 표 2 참조)과 같이 계산합니다. 제 3 관 특약의 성립과 유지 제 7 조 특약의 성립 ① 이 특약은 계약자의 청약(請約)과 회사의 승낙(承諾)으로 전환전계약에 부가하여 이루어 집니다. ② 계약자는 이 특약을 청약할 때 종신연금형, 확정기간연금형 또는 상속연금형 중 한 가지를 선택하여 370 / 462"
3,genai-demo-index-v1,5.091200,간병치매보험_20240101~,"금융소비자 중 대통령령으로 정하는 자가 일반금융소비자와 같은 대우를 받겠다는 의사를 금융상 품판매업자 또는 금융상품자문업자(이하 “금융상품판매업자등”이라 한다)에게 서면으로 통지하는 경우 금융상품판매업자등은 정당한 사유가 있는 경우를 제외하고는 이에 동의하여야 하며, 금융상 품판매업자등이 동의한 경우에는 해당 금융소비자는 일반금융소비자로 본다. 가. 국가 나. 「한국은행법」에 따른 한국은행 다. 대통령령으로 정하는 금융회사 라. 「자본시장과 금융투자업에 관한 법률」 제9조제15항제3호에 따른 주권상장법인(투자성 상품 중 대통령령으로 정하는 금융상품계약체결등을 할 때에는 전문금융소비자와 같은 대우를 받겠다는 의사를 금융상품판매업자등에게 서면으로 통지하는 경우만 해당한다) 마. 그 밖에 금융상품의 유형별로 대통령령으로 정하는 자 10. “일반금융소비자”란 전문금융소비자가 아닌 금융소비자를 말한다. 제42조(소액분쟁사건에 관한 특례) 조정대상기관은 다음 각 호의 요건 모두를 총족하는 분쟁사건(이하 “소액분쟁사건”이라 한다)에 대하 여 조정절차가 개시된 경우에는 제36조제6항에 따라 조정안을 제시받기 전에는 소를 제기할 수 없다. 다만, 제36조제3항에 따라 서면통지를 받거나 제36조제5항에서 정한 기간 내에 조정안을 제시받지 못 한 경우에는 그러하지 아니하다. 1. 일반금융소비자가 신청한 사건일 것 2. 조정을 통하여 주장하는 권리나 이익의 가액이 2천만원 이내에서 대통령령으로 정하는 금액 이하 일 것 제47조(위법계약의 해지) ① 금융소비자는 금융상품판매업자등이 제17조제3항, 제18조제2항, 제19조제1항ㆍ제3항, 제20조제1항 또는 제21조를 위반하여 대통령령으로 정하는 금융상품에 관한 계약을 체결한 경우 5년 이내의 대 통령령으로 정하는 기간 내에 서면등으로 해당 계약의 해지를 요구할 수 있다. 이 경우 금융상품 판매업자등은 해지를 요구받은 날부터 10일 이내에 금융소비자에게 수락여부를 통지하여야 하며, 거절할 때에는 거절사유를 함께 통지하여야 한다. ② 금융소비자는 금융상품판매업자등이 정당한 사유 없이 제1항의 요구를 따르지 않는 경우 해당 계 약을 해지할 수 있다. ③ 제1항 및 제2항에 따라 계약이 해지된 경우 금융상품판매업자등은 수수료, 위약금 등 계약의 해지 와 관련된 비용을 요구

## 7. 벡터 검색 (knn 검색)을 활용한 검색 - Semantic Search

- query 를 제공해서 실제로 유사한 내용이 검색이 되는지를 확인 합니다.

In [31]:
def query_semantic(vector, filter=[], k=5):
    QUERY_TEMPLATE = {
        "size": k,
        "query": {                    
            "knn": {
                "vector_field": {
                    "vector": vector,
                    "k": k 
                }
            }           
        }
    }
    return QUERY_TEMPLATE

In [32]:
print("<<사용자 입력 쿼리 문장>>: ", query)

response = os_client.search(
    body=query_semantic(llm_emb.embed_query(query)),
    index=index_name
)
result_to_dataframe(response)

<<사용자 입력 쿼리 문장>>:  금리연동형 상품은 무엇인가요?


,index_name,score,type,text
0,genai-demo-index-v1,0.004500,일시납종신보험_1912-061~072_20230916~,"【 별표 1 】 연금 지급기준표 급부명칭 지 급 내 용 지급사유 금 액 이 특약의 보험료를 기준으로 피보험 자가 생존한 기간동안(10년보증, 20년 보증, 100세보증) 보험료 및 해약환급 연금개시일 이후 피보험자가 매년 종신연금 금 산출방법서(이하 ‘산출방법서’라 보험계약 해당일에 살아있을 경우 합니다)에서 정한 방법에 따라 연금 액을 분할 계산하여 매년 보험계약 연금 해당일에 지급 연금개시일 이후 피보험자가 「’일상생활장해보장개시일’ 이후에 ‘일상생활장해상태’로 최종 종신연금 연금액의 1배를 진단 확정 LTC연금 진단확정되거나 ‘중증(重症)치매보 일을 포함하여 매년 진단 확정일에 장개시일’ 이후에 ‘중증(重症)치매 지급(최고 10회한도) 상태’로 최종 진단확정」되고 매년 진단 확정일에 살아있을 경우 주) 1. ‘이 특약의 보험료’란 전환전계약의 사업방법서에서 정한 바에 따라 전환전계약의 지급금의 전부 또는 일부를 말합니다. 2. 이 특약의 공시이율은 사업방법서에서 정한 방법에 따라 회사가 매월 1일 결정하며, 공시이율의 최저보증이율은 계약일로부터 3년 이내에는 연복리 1.0%, 3년초과 5년이내에는 연복리 0.75%, 5 년을 초과하는 경우에는 연복리 0.5%를 적용합니다. 3. 연금액의 계산은 이 특약의 보험료를 기준으로 이 특약의 공시이율을 적용하여 산출방법서에서 정한 방법에 따라 계산되며, 공시이율이 변경되면 실제 지급되는 연금액도 변경됩니다. 4. LTC발생시 ‘일상생활장해상태’ 또는 ‘중증(重症)치매상태’ 중 최초로 발생한 지급사유에 한하여 10회를 최고 한도로 하여 LTC연금을 지급합니다. 5. 종신연금의 100세보증은 피보험자 나이를 기준으로 합니다. 6. 종신연금의 경우 보증지급기간(10년, 20년, 100세 보증) 동안 지급되지 않은 종신연금액을 산출 방법서에 따라 공시이율로 할인하여 일시금으로 선지급 받을 수 있으며, 피보험자가 사망하는 경우 또한 같습니다. 7. 연금은 매월, 매3개월, 매6개월로 분할하여 지급받을 수 있습니다. 8. 해당월의 보험계약 해당일이 없는 경우에는 해당월의 말일을 보험계약 해당일로 합니다. 9. 보장개시일: 이 특약의 보장개시일은 이 특약의 계약자가 신청한 연금전환일로 하며, 보장개시 일을 보험계약일로 봅니다. 10. 일상생활장해보장개시일: 연금전환일부터 그 날을 포함하여 90일이 지난날의 다음 날로 합니다. 다만, 보험기간 중 발생한 재해로 인하여 ‘일상생활장해상태’가 발생한 경우에는 연금전환일을 보장개시일로 합니다. 11. 중증(重症)치매보장개시일: 연금전환일부터 그 날을 포함하여 만 2년이 지난 날의 다음 날로 합니다. 다만, 질병으로 인한 ‘중증(重症)치매상태’가 없는 상태에서 보험기간 중 발생한 재해로 인한 뇌의 손상으로 인하여 ‘중증(重症)치매상태’가 발생한 경우에는 연금전환일을 보장개시일 로 합니다. 112 / 235"
1,genai-demo-index-v1,0.004500,일시납종신보험_20240101~,"【 별표 1 】 연금 지급기준표 급부명칭 지 급 내 용 지급사유 금 액 이 특약의 보험료를 기준으로 피보험 자가 생존한 기간동안(10년보증, 20년 보증, 100세보증) 보험료 및 해약환급 연금개시일 이후 피보험자가 매년 종신연금 금 산출방법서(이하 ‘산출방법서’라 보험계약 해당일에 살아있을 경우 합니다)에서 정한 방법에 따라 연금 액을 분할 계산하여 매년 보험계약 연금 해당일에 지급 연금개시일 이후 피보험자가 「’일상생활장해보장개시일’ 이후에 ‘일상생활장해상태’로 최종 종신연금 연금액의 1배를 진단 확정 LTC연금 진단확정되거나 ‘중증(重症)치매보 일을 포함하여 매년 진단 확정일에 장개시일’ 이후에 ‘중증(重症)치매 지급(최고 10회한도) 상태’로 최종 진단확정」되고 매년 진단 확정일에 살아있을 경우 주) 1. ‘이 특약의 보험료’란 전환전계약의 사업방법서에서 정한 바에 따라 전환전계약의 지급금의 전부 또는 일부를 말합니다. 2. 이 특약의 공시이율은 사업방법서에서 정한 방법에 따라 회사가 매월 1일 결정하며, 공시이율의 최저보증이율은 계약일로부터 3년 이내에는 연복리 1.0%, 3년초과 5년이내에는 연복리 0.75%, 5 년을 초과하는 경우에는 연복리 0.5%를 적용합니다. 3. 연금액의 계산은 이 특약의 보험료를 기준으로 이 특약의 공시이율을 적용하여 산출방법서에서 정한 방법에 따라 계산되며, 공시이율이 변경되면 실제 지급되는 연금액도 변경됩니다. 4. LTC발생시 ‘일상생활장해상태’ 또는 ‘중증(重症)치매상태’ 중 최초로 발생한 지급사유에 한하여 10회를 최고 한도로 하여 LTC연금을 지급합니다. 5. 종신연금의 100세보증은 피보험자 나이를 기준으로 합니다. 6. 종신연금의 경우 보증지급기간(10년, 20년, 100세 보증) 동안 지급되지 않은 종신연금액을 산출 방법서에 따라 공시이율로 할인하여 일시금으로 선지급 받을 수 있으며, 피보험자가 사망하는 경우 또한 같습니다. 7. 연금은 매월, 매3개월, 매6개월로 분할하여 지급받을 수 있습니다. 8. 해당월의 보험계약 해당일이 없는 경우에는 해당월의 말일을 보험계약 해당일로 합니다. 9. 보장개시일: 이 특약의 보장개시일은 이 특약의 계약자가 신청한 연금전환일로 하며, 보장개시 일을 보험계약일로 봅니다. 10. 일상생활장해보장개시일: 연금전환일부터 그 날을 포함하여 90일이 지난날의 다음 날로 합니다. 다만, 보험기간 중 발생한 재해로 인하여 ‘일상생활장해상태’가 발생한 경우에는 연금전환일을 보장개시일로 합니다. 11. 중증(重症)치매보장개시일: 연금전환일부터 그 날을 포함하여 만 2년이 지난 날의 다음 날로 합니다. 다만, 질병으로 인한 ‘중증(重症)치매상태’가 없는 상태에서 보험기간 중 발생한 재해로 인한 뇌의 손상으로 인하여 ‘중증(重症)치매상태’가 발생한 경우에는 연금전환일을 보장개시일 로 합니다. 112 / 249"
2,genai-demo-index-v1,0.004400,간병치매보험_20231209~_1,"보장한도 이 보험에는 ‘보험금의 지급 횟수 등의 한도’가 있는 [보장한도]가 적용되는 담보가 포함되어 있습니다. 구분 담보명 보장한도 주계약 중증 치매진단자금 최초 1회에 한해 보장 치매(CDR1점이상) 경도이상 치매진단자금 최초 1회에 한해 보장 보장특약 치매(CDR2점이상) 보장특약 중등도이상 치매진단자금 최초 1회에 한해 보장 [종속/독립] 치매(CDR3점이상) 중증 치매진단자금 최초 1회에 한해 보장 보장특약 치매(CDR5점) 말기 치매진단자금 최초 1회에 한해 보장 보장특약 최초 1회에 한해 보장, 치매(CDR1점이상) 경도이상 치매간병자금 최초 3년 보증지급, 간병자금특약 생존시 최대 5년(60회) 치매(CDR2점이상) 최초 1회에 한해 보장, 간병자금특약 중등도이상 치매간병자금 최초 3년 보증지급, (종신지급) 생존시 종신지급 치매(CDR3점이상) 최초 1회에 한해 보장, 간병자금특약 중증 치매간병자금 최초 3년 보증지급, (종신지급) 생존시 종신지급 치매(CDR5점) 최초 1회에 한해 보장, 간병자금특약 말기 치매간병자금 최초 3년 보증지급, (종신지급) 생존시 종신지급 장기요양(1~인지지원등급) 장기요양상태 판정자금 최초 1회에 한해 보장 보장특약 장기요양(1~5등급) 경도이상 장기요양상태 판정자금 최초 1회에 한해 보장 보장특약 장기요양(1~2등급) 중증 장기요양상태 판정자금 최초 1회에 한해 보장 보장특약 22 / 448"
3,genai-demo-index-v1,0.004400,간병치매보험_20240101~,"보장한도 이 보험에는 ‘보험금의 지급 횟수 등의 한도’가 있는 [보장한도]가 적용되는 담보가 포함되어 있습니다. 구분 담보명 보장한도 주계약 중증 치매진단자금 최초 1회에 한해 보장 치매(CDR1점이상) 경도이상 치매진단자금 최초 1회에 한해 보장 보장특약 치매(CDR2점이상) 보장특약 중등도이상 치매진단자금 최초 1회에 한해 보장 

이전에 어휘분석으로 검색한 결과는 아래와 같습니다. 위의 표와 비교해보시길 바랍니다.

In [33]:
print("<<사용자 입력 쿼리 문장>>: ", query)
result_to_dataframe(response_lexical_only)

<<사용자 입력 쿼리 문장>>:  금리연동형 상품은 무엇인가요?


,index_name,score,type,text
0,genai-demo-index-v1,18.470600,일시납종신보험_1912-061~072_20230916~,"Ⅱ. 약관 요약서 1. 보험계약의 개요 가입하신 보험상품의 보장내용과 특징을 확인해보세요. 한화생명 The스마트한 일시납종신보험 무배당 [보증비용부과형] 상품의 종목 : 보장성·종신보험 (1) 상품의 주요특징 예금자보호 금리연동형 예금자보호법에 따라 보험회사의 자산운용이익률, 1인당 최고 5천만원까지 시장금리 등에 따라 적립액 보장합니다. 다만, 보험계약자 적용이율이 변동되는 상품입 및 보험료 납부자가 법인인 니다. 경우에는 보호하지 않습니다. 보장성보험 한화생명 무배당 The스마트한 사망, 질병, 계약자에게 일시납종신보험 상해 등의 위험을 배당을 하지 않는 무배당 보장하는 보장성 상품입니다. 보험입니다. [보증비용부과형] 최저보증이율 해약환급금 보증 (해약환급금 보증형 가입시) 금리가 하락하더라도 적립시 적용되는 공시이율을 최저해약환급금은 회사가 정한 시산에 따라 향후 공시이율에 관계없이 일정 비율로 최저보증합니다. 계약 해지 시 지급하는 최저한도의 해약환급 금으로서, 예정적립액과 동일합니다. 15 / 235"
1,genai-demo-index-v1,18.470600,일시납종신보험_20240101~,"Ⅱ. 약관 요약서 1. 보험계약의 개요 가입하신 보험상품의 보장내용과 특징을 확인해보세요. 한화생명 The스마트한 일시납종신보험 무배당 [보증비용부과형] 상품의 종목 : 보장성·종신보험 (1) 상품의 주요특징 예금자보호 금리연동형 예금자보호법에 따라 보험회사의 자산운용이익률, 1인당 최고 5천만원까지 시장금리 등에 따라 적립액 보장합니다. 다만, 보험계약자 적용이율이 변동되는 상품입 및 보험료 납부자가 법인인 니다. 경우에는 보호하지 않습니다. 보장성보험 한화생명 무배당 The스마트한 사망, 질병, 계약자에게 일시납종신보험 상해 등의 위험을 배당을 하지 않는 무배당 보장하는 보장성 상품입니다. 보험입니다. [보증비용부과형] 최저보증이율 해약환급금 보증 (해약환급금 보증형 가입시) 금리가 하락하더라도 적립시 적용되는 공시이율을 최저해약환급금은 회사가 정한 시산에 따라 향후 공시이율에 관계없이 일정 비율로 최저보증합니다. 계약 해지 시 지급하는 최저한도의 해약환급 금으로서, 예정적립액과 동일합니다. 15 / 249"
2,genai-demo-index-v1,15.899300,일시납종신보험_1912-061~072_20230916~,"(2) 소비자가 반드시 알아두어야 할 상품의 주요 특성 보장성보험 ① 이 보험은 사망 보장을 주목적으로 하는 보장성보험이며, 저축이나 연금수령을 목적으로 가입하시기에 적합하지 않습니다. ② 만기 또는 중도해지 시 납입한 보험료보다 환급금이 적을 수 있습니다. ③ 사망보험금의 일부 또는 전부를 해지하여 연금으로 전환할 수 있으나, 처음부터 연금보험에 가입한 경우보다 연금액이 적습니다. [민원 사례] A씨는 저축을 주된 목적으로 종신보험을 가입하였으나 알고보니 사망 및 질병 보장을 주목적으로 하는 보장성보험임에 불만 ▶ 종신보험 등 보장성보험은 저축이나 노후에 연금 수령을 목적으로 가입하시기에는 적합하지 않으며 또한 중도에 해지하면 손해를 볼 수 있음 금리연동형보험 ① 이 보험의 적립액 산출에 적용되는 이율은 매월 변동됩니다. ② 동 이율은 납입한 주계약(또는 적립) 보험료에서 계약체결·유지관리에 필요한 경비 및 위험보장을 위한 보험료를 차감한 금액에 대해서만 적용됩니다. ③ 이 보험의 최저보증이율*은 3년이내에는 연복리 1.0%, 3년초과 5년이내에는 연복리 0.75%, 5년을 초과하는 경우에는 연복리 0.5%입니다. *자산운용이익률, 시장금리가 하락하는 경우 회사가 보증하는 적용이율의 최저한도 유니버셜보험 이 보험은 적립액 인출 등을 하는 경우 계약이 조기에 실효될 수 있습니다. * 해약환급금에서 월대체보험료(계약 유지관리 등에 소요되는 경비, 위험 보장을 위한 보험료)를 매 월 공제 [민원 사례] A씨는 일시납보험료를 하면 보장을 계속 받을 수 있다고 생각하고 ○○○○ 유니버설종신보험에 가입하였으나, 4년이 지난 후 보험회사에서 보험료 납입을 독촉하는 안내를 받음 ▶ 보험회사는 유니버설보험의 경우 기본보험료 납입 이후 중도인출 등으로 인해, 주계약 해약환급금에서 월대체보험료를 충당할 수 없게 된 경우 계약이 해지될 수 있고, 계약을 유지하기 위해서는 보험료 추가 납입이 필요함을 안내 예금자보호제도에 ① 이 보험은 예금자보호법에 따라 예금보험공사가 보호합니다. 관한 사항 ② 예금자보호 한도는 본 보험회사에 있는 귀하의 모든 예금보호 대상 금융상품의 해약환급금(또는 만기시 보험금이나 사고보험금)과 기타 지급금을 합하여 1인당 “최고 5천만원”이며, 5천만원을 초과하는 나머지 금액은 보호하지 않습니다. ③ 또한, 보험계약자 및 보험료 납부자가 법인이면 보호되지 않습니다. 19 / 235"
3,genai-demo-index-v1,14.382200,일시납종신보험_20240101~,"(2) 소비자가 반드시 알아두어야 할 상품의 주요 특성 보장성보험 ① 이 보험은 사망 보장을 주목적으로 하는 보장성보험이며, 저축이나 연금수령을 목적으로 가입하시기에 적합하지 않습니다. ② 만기 또는 중도해지 시 납입한 보험료보다 환급금이 적을 수 있습니다. ③ 사망보험금의 일부 또는 전부를 해지하여 연금으로 전환할 수 있으나, 처음부터 연금보험에 가입한 경우보다 연금액이 적습니다. [민원 사례] A씨는 저축을 주된 목적으로 종신보험을 가입하였으나 알고보니 사망 및 질병 보장을 주목적으로 하는 보장성보험임에 불만 ▶ 종신보험 등 보장성보험은 저축이나 노후에 연금 수령을 목적으로 가입하시기에는 적합하지 않으며 또한 중도에 해지하면 손해를 볼 수 있음 금리연동형보험 ① 이 보험의 적립액 산출에 적용되는 이율은 매월 변동됩니다. ② 동 이율은 납입한 주계약(또는 적립) 보험료에서 계약체결·유지관리에 필요한 경비 및 위험보장을 위한 보험료를 차감한 금액에 대해서만 적용됩니다. ③ 이 보험의 최저보증이율*은 3년이내에는 연복리 1.0%, 3년초과 5년이내에는 연복리 0.75%, 5년을 초과하는 경우에는 연복리 0.5%입니다. *자산운용이익률, 시장금리가 하락하는 경우 회사가 보증하는 적용이율의 최저한도 유니버셜보험 이 보험은 적립액 인출 등을 하는 경우 계약이 조기에 실효될 수 있습니다. * 해약환급금에서 월대체보험료(계약 유지관리 등에 소요되는 경비, 위험 보장을 위한 보험료)를 매 월 공제 [민원 사례] A씨는 일시납보험료를 하면 보장을 계속 받을 수 있다고 생각하고 ○○○○ 유니버설종신보험에 가입하였으나, 4년이 지난 후 보험회사에서 보험료 납입을 독촉하는 안내를 받음 ▶ 보험회사는 유니버설보험의 경우 기본보험료 납입 이후 중도인출 등으로 인해, 주계약 해약환급금에서 월대체보험료를 충당할 수 없게 된 경우 계약이 해지될 수 있고, 계약을 유지하기 위해서는 보험료 추가 납입이 필요함을 안내 예금자보호제도에 ① 이 보험은 예금자보호법에 따라 예금보험공사가 보호합니다. 관한 사항 ② 예금자보호 한도는 본 보험회사에 있는 귀하의 모든 예금보호 대상 금융상품의 해약환급금(또는 만기시 보험금이나 사고보험금)과 기타 지급금을 합하여 1인당 “최고 5천만원”이며, 5천만원을 초과하는 나머지 금액은 보호하지 않습니다. ③ 또한, 보험계약자 및 보험료 납부자가 법인이면 보호되지 않습니다. 19 / 249"
4,genai-demo-index-v1,11.434800,간병치매보험_20231209~_1,"Ⅱ. 약관 요약서 1. 보험계약의 개요 가입하신 보험상품의 보장내용과 특징을 확인해보세요. 한화생명 The실속있는 간병치매보험 무배당 상품의 종목: 보장성보험, 치매보험 (1) 상품의 주요특징 (1) 상품의 주요특징 금리확정형 예금자보호 보험료 및 적립금 산출에 예금자보호법에 따라 적용되는 이율이 1인당 최고 5천만원까지 보험기간 동안 고정되어 보장합니다. 변동이 없습니다. 보장성보험 무배당 사망, 

## 8. LangChain을 이용한 Question & Answer

- langchain의 similarity_search_with_score API를 활용하는 방법
    - [API: similarity_search_with_score](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html#langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.similarity_search)


In [34]:
from langchain.chains.question_answering import load_qa_chain

In [35]:
results = vector_db.similarity_search_with_score(
    query=query,
    k=5,
    search_type="approximate_search",
    boolean_filter={
        "bool": {
            "filter": []
        }
    }
)

In [36]:
[res[0].page_content for res in results[:3]]

['【 별표 1 】 연금 지급기준표 급부명칭 지 급 내 용 지급사유 금 액 이 특약의 보험료를 기준으로 피보험 자가 생존한 기간동안(10년보증, 20년 보증, 100세보증) 보험료 및 해약환급 연금개시일 이후 피보험자가 매년 종신연금 금 산출방법서(이하 ‘산출방법서’라 보험계약 해당일에 살아있을 경우 합니다)에서 정한 방법에 따라 연금 액을 분할 계산하여 매년 보험계약 연금 해당일에 지급 연금개시일 이후 피보험자가 「’일상생활장해보장개시일’ 이후에 ‘일상생활장해상태’로 최종 종신연금 연금액의 1배를 진단 확정 LTC연금 진단확정되거나 ‘중증(重症)치매보 일을 포함하여 매년 진단 확정일에 장개시일’ 이후에 ‘중증(重症)치매 지급(최고 10회한도) 상태’로 최종 진단확정」되고 매년 진단 확정일에 살아있을 경우 주) 1. ‘이 특약의 보험료’란 전환전계약의 사업방법서에서 정한 바에 따라 전환전계약의 지급금의 전부 또는 일부를 말합니다. 2. 이 특약의 공시이율은 사업방법서에서 정한 방법에 따라 회사가 매월 1일 결정하며, 공시이율의 최저보증이율은 계약일로부터 3년 이내에는 연복리 1.0%, 3년초과 5년이내에는 연복리 0.75%, 5 년을 초과하는 경우에는 연복리 0.5%를 적용합니다. 3. 연금액의 계산은 이 특약의 보험료를 기준으로 이 특약의 공시이율을 적용하여 산출방법서에서 정한 방법에 따라 계산되며, 공시이율이 변경되면 실제 지급되는 연금액도 변경됩니다. 4. LTC발생시 ‘일상생활장해상태’ 또는 ‘중증(重症)치매상태’ 중 최초로 발생한 지급사유에 한하여 10회를 최고 한도로 하여 LTC연금을 지급합니다. 5. 종신연금의 100세보증은 피보험자 나이를 기준으로 합니다. 6. 종신연금의 경우 보증지급기간(10년, 20년, 100세 보증) 동안 지급되지 않은 종신연금액을 산출 방법서에 따라 공시이율로 할인하여 일시금으로 선지급 받을 수 있으며, 피보험자가 사망하는 경우 또한 같습니다. 7. 연금은 매월, 매3개월, 매6개월로 분할하여 지급받을 수 있습니다. 8.

### 사용자 정의 가능한 옵션
이제 벡터 저장소가 준비되었으므로 질문을 시작할 수 있습니다.

Vector Store를 둘러싸서 LLM 입력을 받는 LangChain에서 제공하는 래퍼를 사용할 수 있습니다.
이 래퍼는 뒤에서 다음 단계를 수행합니다.
- 질문을 입력합니다.
- 질문 임베딩 생성
- 관련 문서 가져오기
- 프롬프트에 문서와 질문을 채워 넣습니다.
- 프롬프트로 모델을 호출하고 사람이 읽을 수 있는 방식으로 답변을 생성합니다.

위 시나리오에서는 질문에 대한 상황 인식 답변을 빠르고 쉽게 얻을 수 있는 방법을 탐색했습니다. 이제 문서를 가져오는 방법을 사용자 정의할 수 있는 [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/Vector_db_qa.html)의 도움으로 더 사용자 정의 가능한 옵션을 살펴보겠습니다. `chain_type` 매개변수를 사용하여 프롬프트에 추가해야 합니다. 또한 검색해야 하는 관련 문서 수를 제어하려면 아래 셀에서 'k' 매개변수를 변경하여 다른 출력을 확인하세요. 많은 시나리오에서 LLM이 답변을 생성하는 데 사용한 소스 문서가 무엇인지 알고 싶을 수 있습니다. LLM 프롬프트의 컨텍스트에 추가된 문서를 반환하는 `return_source_documents`를 사용하여 출력에서 ​​해당 문서를 가져올 수 있습니다. 'RetrievalQA'를 사용하면 모델에 특정한 사용자 정의 [프롬프트 템플릿](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html)을 제공할 수도 있습니다.

참고: 이 예에서는 Amazon Bedrock에서 LLM으로 Anthropic Claude를 사용하고 있습니다. 이 특정 모델은 입력이 'Human:' 아래에 제공되고 모델이 'Assistant:' 다음에 출력을 생성하도록 요청되는 경우 가장 잘 수행됩니다. 아래 셀에는 LLM이 기본 상태를 유지하고 컨텍스트 외부에서 응답하지 않도록 프롬프트를 제어하는 ​​방법의 예가 나와 있습니다.

#### [[REF] Using langchain for Question Answering on Own Data](https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed)

In [37]:
from langchain.schema import BaseRetriever
from typing import Any, Dict, List, Optional, List, Tuple
from langchain.callbacks.manager import CallbackManagerForRetrieverRun

# lexical(keyword) search based (using Amazon OpenSearch)
class OpenSearchLexicalSearchRetriever(BaseRetriever):
    os_client: Any
    index_name: str
    k = 3
    filter = []

    def normalize_search_results(self, search_results):
        hits = (search_results["hits"]["hits"])
        max_score = float(search_results["hits"]["max_score"])
        for hit in hits:
            hit["_score"] = float(hit["_score"]) / max_score
        search_results["hits"]["max_score"] = hits[0]["_score"]
        search_results["hits"]["hits"] = hits
        return search_results

    def update_search_params(self, **kwargs):
        self.k = kwargs.get("k", 3)
        self.filter = kwargs.get("filter", [])
        self.index_name = kwargs.get("index_name", self.index_name)

    def _reset_search_params(self, ):
        self.k = 3
        self.filter = []
        
    def query_lexical(self, query, filter=[], k=5):
        QUERY_TEMPLATE = {
            "size": k,
            "query": {
                "bool": {
                    "must": [
                        {
                            "match": {
                                "text": {
                                    "query": query,
                                    "operator":  "or"
                                }
                            }
                        }
                    ],
                    "filter": filter
                }
            }
        }
        
        if len(filter) > 0:
            QUERY_TEMPLATE["query"]["bool"]["filter"].extend(filter)
            
        return QUERY_TEMPLATE
    

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun) -> List[Document]:
        
        query = self.query_lexical(
            query=query,
            filter=self.filter,
            k=self.k
        )

        # print ("lexical search query: ")
        # print(query)
        
        search_results = self.os_client.search(
            body=query,
            index=self.index_name
        )

        results = []
        if search_results["hits"]["hits"]:
            search_results = self.normalize_search_results(search_results)
            for res in search_results["hits"]["hits"]:

                metadata = res["_source"]["metadata"]
                metadata["id"] = res["_id"]

                doc = Document(
                    page_content=res["_source"]["text"],
                    metadata=metadata
                )
                results.append((doc))

        self._reset_search_params()

        return results[:self.k]


In [38]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
# from utils.rag import run_RetrievalQA, show_context_used

In [39]:
prompt_template = """
\n\nHuman: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

\n\nAssistant:"""


PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [40]:
chain = load_qa_chain(
    llm=llm_text,
    chain_type="stuff",
    prompt=PROMPT,
    verbose=True
)

In [41]:
boolean_filter = []
boolean_filter = [
    {"term": {"metadata.source": "한화생명 The스마트한 일시납종신보험_20240101~.pdf"}},
    {"term": {"metadata.type": "일시납종신보험_20240101~"}},
]

In [42]:
opensearch_lexical_retriever = OpenSearchLexicalSearchRetriever(
    os_client=os_client,
    index_name=index_name,
    k=3,
    filter=boolean_filter
)

In [43]:
answer = chain.invoke(
    {
        "input_documents": opensearch_lexical_retriever.get_relevant_documents(query), 
        "question": query
    }, 
    # return_only_outputs=True
)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Ⅱ. 약관 요약서 1. 보험계약의 개요 가입하신 보험상품의 보장내용과 특징을 확인해보세요. 한화생명 The스마트한 일시납종신보험 무배당 [보증비용부과형] 상품의 종목 : 보장성·종신보험 (1) 상품의 주요특징 예금자보호 금리연동형 예금자보호법에 따라 보험회사의 자산운용이익률, 1인당 최고 5천만원까지 시장금리 등에 따라 적립액 보장합니다. 다만, 보험계약자 적용이율이 변동되는 상품입 및 보험료 납부자가 법인인 니다. 경우에는 보호하지 않습니다. 보장성보험 한화생명 무배당 The스마트한 사망, 질병, 계약자에게 일시납종신보험 상해 등의 위험을 배당을 하지 않는 무배당 보장하는 보장성 상품입니다. 보험입니다. [보증비용부과형] 최저보증이율 해약환급금 보증 (해약환급금 보증형 가입시) 금리가 하락하더라도 적립시 적용되는 공시이율을 최저해약환급금은 회사가 정한 시산에 따라 향후 공시이율에 관계없이 일정 비율로 최저보증합니다. 계약 해지 시 지급하는 최저한도의 해약환급 금으로서, 예정적립액과 동일합니다. 15 / 249

(2) 소비자가 반드시 알아두어야 할 상품의 주요 특성 보장성보험 ① 이 보험은 사망 보장을 주목적으로 하는 보장성보험이며, 저축이나 연금수령을 목적으로 가입하시기에 적합하지 않습니다. ② 만기 또는 중도해지 시 납입한 보험료보다 환급금이 적을 수 있습니다. ③ 사망보험금의 일부 또는 전부를 해지하여 연금으로 전환할 수 있

In [44]:
opensearch_semantic_retriever = vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
        "boolean_filter": boolean_filter
    }
)

In [45]:
answer = chain.invoke(
    {
        "input_documents": opensearch_semantic_retriever.get_relevant_documents(query), 
        "question": query
    }, 
    # return_only_outputs=True
)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

【 별표 1 】 연금 지급기준표 급부명칭 지 급 내 용 지급사유 금 액 이 특약의 보험료를 기준으로 피보험 자가 생존한 기간동안(10년보증, 20년 보증, 100세보증) 보험료 및 해약환급 연금개시일 이후 피보험자가 매년 종신연금 금 산출방법서(이하 ‘산출방법서’라 보험계약 해당일에 살아있을 경우 합니다)에서 정한 방법에 따라 연금 액을 분할 계산하여 매년 보험계약 연금 해당일에 지급 연금개시일 이후 피보험자가 「’일상생활장해보장개시일’ 이후에 ‘일상생활장해상태’로 최종 종신연금 연금액의 1배를 진단 확정 LTC연금 진단확정되거나 ‘중증(重症)치매보 일을 포함하여 매년 진단 확정일에 장개시일’ 이후에 ‘중증(重症)치매 지급(최고 10회한도) 상태’로 최종 진단확정」되고 매년 진단 확정일에 살아있을 경우 주) 1. ‘이 특약의 보험료’란 전환전계약의 사업방법서에서 정한 바에 따라 전환전계약의 지급금의 전부 또는 일부를 말합니다. 2. 이 특약의 공시이율은 사업방법서에서 정한 방법에 따라 회사가 매월 1일 결정하며, 공시이율의 최저보증이율은 계약일로부터 3년 이내에는 연복리 1.0%, 3년초과 5년이내에는 연복리 0.75%, 5 년을 초과하는 경우에는 연복리 0.5%를 적용합니다. 3. 연금액의 계산은 이 특약의 보험료를 기준으로 이 특약의 공시이율을 적용하여 산출방법서에서 정한 방법에 따라 계산되며, 공시이율이 변경되면 실제 지급되는

## 9. OpenSearch Hybrid 검색

OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) "Vector 서치" 하여 스코어를 얻은 후에 표준화를 하여 스코어를 구함. 
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (2) Keyword 서치도 동일하게 함.
- (3) Reciprocal Rank Fusion (RRF) 기반 Re-rank
    - Paper: https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    - Desc: https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d
    - **RRF의 경우 score가 아닌 ranking 정보를 활용, 때문에 score normalization이 필요 없음**

RRF는 langchain에서 "Ensemble Retriever" 이름으로 api를 제공합니다. 
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble


### Ensemble retriever 정의
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble
- RRF 방식만 지원
- Rank constant (param "c")
    - This value determines how much influence documents in individual result sets per query have over the final ranked result set. A higher value indicates that lower ranked documents have more influence. This value must be greater than or equal to 1. Defaults to 60.
    - 숫자 높을 수록 낮은 랭크의 문서가 더 중요시 된다

In [46]:
from langchain.retrievers import EnsembleRetriever

In [47]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[opensearch_lexical_retriever, opensearch_semantic_retriever],
    weights=[0.5, 0.5],
    c=100,
    k=5
)

In [48]:
%%time
answer = chain.invoke(
    {
        "input_documents": ensemble_retriever.get_relevant_documents(query), 
        "question": query
    }
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Ⅱ. 약관 요약서 1. 보험계약의 개요 가입하신 보험상품의 보장내용과 특징을 확인해보세요. 한화생명 The스마트한 일시납종신보험 무배당 [보증비용부과형] 상품의 종목 : 보장성·종신보험 (1) 상품의 주요특징 예금자보호 금리연동형 예금자보호법에 따라 보험회사의 자산운용이익률, 1인당 최고 5천만원까지 시장금리 등에 따라 적립액 보장합니다. 다만, 보험계약자 적용이율이 변동되는 상품입 및 보험료 납부자가 법인인 니다. 경우에는 보호하지 않습니다. 보장성보험 한화생명 무배당 The스마트한 사망, 질병, 계약자에게 일시납종신보험 상해 등의 위험을 배당을 하지 않는 무배당 보장하는 보장성 상품입니다. 보험입니다. [보증비용부과형] 최저보증이율 해약환급금 보증 (해약환급금 보증형 가입시) 금리가 하락하더라도 적립시 적용되는 공시이율을 최저해약환급금은 회사가 정한 시산에 따라 향후 공시이율에 관계없이 일정 비율로 최저보증합니다. 계약 해지 시 지급하는 최저한도의 해약환급 금으로서, 예정적립액과 동일합니다. 15 / 235

【 별표 1 】 연금 지급기준표 급부명칭 지 급 내 용 지급사유 금 액 이 특약의 보험료를 기준으로 피보험 자가 생존한 기간동안(10년보증, 20년 보증, 100세보증) 보험료 및 해약환급 연금개시일 이후 피보험자가 매년 종신연금 금 산출방법서(이하 ‘산출방법서’라 보험계약 해당일에 살아있을 경우 합니다)에서 정한 방법에 